## Summary

---

## Imports

In [ ]:
import os
import subprocess
import shlex
import tempfile
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import lightgbm
from scipy import stats

In [ ]:
pd.set_option("max_columns", 1000)

## Paramters

In [ ]:
if "DATAPKG_OUTPUT_DIR" in os.environ:
    OUTPUT_DIR = Path(os.getenv("DATAPKG_OUTPUT_DIR")).joinpath("elaspic-v2").resolve()
else:
    OUTPUT_DIR = NOTEBOOK_DIR.parent
OUTPUT_DIR.mkdir(exist_ok=True)

OUTPUT_DIR

In [ ]:
if (slurm_tmpdir := os.getenv("SLURM_TMPDIR")) is not None:
    os.environ["TMPDIR"] = slurm_tmpdir
    
print(tempfile.gettempdir())

In [ ]:
datasets = [
#     "elaspic-training-set-core",
#     "protherm-dagger-core",
#     "rocklin-2017-core",
    "elaspic-training-set-interface",
]

In [ ]:
feature_generators = [
    "02_run_rosetta_ddg",
    "02_run_proteinsolver",
    "02_run_protbert",
]

### Load data

In [ ]:
def expand_mutations(df):
    results =[]
    for row in df.itertuples():
        for idx in range(len(row.mutation)):
            row_mut = {
                "unique_id": row.unique_id,
                "dataset": row.dataset,
                'name': row.name,
                "mutation": row.mutation[idx],
                "effect": row.effect[idx],
                "effect_type": row.effect_type,
            }
            for column in ["provean_score", "foldx_score", "elaspic_score"]:
                if hasattr(row, column):
                    row_mut[column] = getattr(row, column)[idx]
            results.append(row_mut)
    return pd.DataFrame(results)

In [ ]:
def add_mutation_complement(df):
    df_in = df

    df = df.copy()
    df["mutation"] = df["mutation"].str[-1] + df["mutation"].str[1:-1] + df["mutation"].str[0]
    for column in ["effect", "provean_score", "foldx_score", "elaspic_score"]:
        if column in df:
            df[column] = -df[column]
    for column in df:
        if column.endswith("_wt"):
            column_mut = column[:-3] + "_mut" 
            df[column], df[column_mut] = df[column_mut].copy(), df[column].copy()
    
    df_out = pd.concat([df_in, df], ignore_index=True)
    return df_out

In [ ]:
tmp_df = pd.DataFrame([
    [0, "M1A", 1.234, "wt score", "mut score"],
    [1, "M2C", -0.05, "wt score 2", "mut score 2"]
], columns=["unique_id", "mutation", "effect", "feature_wt", "feature_mut"])

tmp2_df = add_mutation_complement(tmp_df)

display(tmp_df)
display(tmp2_df)

In [ ]:
for dataset_name in datasets:
    input_file = OUTPUT_DIR.joinpath("01_load_data", f"{dataset_name}.parquet")
    pfile = pq.ParquetFile(input_file)
    task_count = pfile.num_row_groups
    df = pfile.read().to_pandas(integer_object_nulls=True)
    expanded_df = (
        add_mutation_complement(expand_mutations(df))
        .drop_duplicates(subset=["unique_id", "mutation"])
        .sort_values(["unique_id", "mutation"])
    )
    sequence_df = df[["unique_id", "protein_sequence", "ligand_sequence"]].drop_duplicates()

    features = {}
    for feature_generator in feature_generators:
        output_dir = OUTPUT_DIR.joinpath(feature_generator)
        feature_dfs = []
        for task_id in range(1, task_count + 1):
            if feature_generator in  ["02_run_rosetta_ddg"]:
                #
                output_file_wt2mut = output_dir.joinpath(f"{dataset_name}-wt2mut-{task_id}-{task_count}.parquet")
                if not output_file_wt2mut.is_file():
                    print(f"File {output_file_wt2mut} is missing. Skipping...")
                    continue
                feature_wt2mut_df = pq.read_table(output_file_wt2mut).to_pandas(integer_object_nulls=True)
                feature_dfs.append(feature_wt2mut_df)
                
                #
                output_file_mut2wt = output_dir.joinpath(f"{dataset_name}-mut2wt-{task_id}-{task_count}.parquet")
                if not output_file_mut2wt.is_file():
                    print(f"File {output_file_mut2wt} is missing. Skipping...")
                    continue
                feature_mut2wt_df = pq.read_table(output_file_mut2wt).to_pandas(integer_object_nulls=True)
                feature_mut2wt_df["unique_id"] = -feature_mut2wt_df["unique_id"]
                feature_dfs.append(feature_mut2wt_df)
            else:
                output_file = output_dir.joinpath(f"{dataset_name}-{task_id}-{task_count}.parquet")
                if not output_file.is_file():
                    print(f"File {output_file} is missing. Skipping...")
                    continue
                feature_df = pq.read_table(output_file).to_pandas(integer_object_nulls=True)
                feature_df = add_mutation_complement(feature_df)
                feature_dfs.append(feature_df)

        feature_df = pd.concat(feature_dfs, ignore_index=True)
        features[feature_generator] = feature_df

### Merge together

In [ ]:
def merge_feature_dfs(feature_dfs):
    def _clean_df(df):
        df = df.copy()
        assert len(df) == len(df[["unique_id", "mutation"]].drop_duplicates())
        for column in ["effect", "effect_type", "provean_score", "foldx_score", "elaspic_score"]:
            if column in df:
                del df[column]
        return df

    df = _clean_df(feature_dfs[0])
    for other_df in feature_dfs[1:]:
        df = df.merge(_clean_df(other_df), how="outer", on=["unique_id", "mutation"])
    return df

In [ ]:
merged_features_df = merge_feature_dfs(list(features.values())).sort_values(["unique_id", "mutation"])
assert merged_features_df["unique_id"].min() >= 0

len(merged_features_df)

In [ ]:
merged_features_df.head()

In [ ]:
merged_features_nonan_df = merged_features_df.dropna()
print(f"Lost {len(merged_features_df) - len(merged_features_nonan_df):,} out of {len(merged_features_df):,} rows due to missing values.")

len(merged_features_nonan_df)

In [ ]:
expanded_df.head()

In [ ]:
final_df = expanded_df.merge(merged_features_nonan_df, on=["unique_id", "mutation"], validate="1:1")
assert len(final_df) == len(merged_features_nonan_df)
print(f"Lost {len(expanded_df) - len(final_df):,} out of {len(expanded_df):,} rows due to missing features.")

In [ ]:
len(final_df) , len(merged_features_nonan_df)

### Feature engineering

In [ ]:
display(final_df.head())
print(len(final_df))

In [ ]:
assert not final_df["foldx_score"].isnull().any()
assert not final_df["effect"].isnull().any()

In [ ]:
final_df["effect_type"].value_counts()

In [ ]:
from scipy import stats

# df = final_df[final_df["effect_type"] == "Deleteriousness class"].dropna()
df = final_df[final_df["effect_type"] == "ΔΔG"]

stats.spearmanr(df["effect"], df["rosetta_bind_dg_change"])
stats.spearmanr(df["effect"], df["rosetta_bind_dg_change"])

### Clustering

In [ ]:
def obtain_clusters(input_sequences, min_seq_id=0.3):
    with tempfile.TemporaryDirectory() as tmp_dir:
        input_dir = Path(tmp_dir, "input")
        input_dir.mkdir()
        
        output_dir = Path(tmp_dir, "output")
        output_dir.mkdir()
        
        scratch_dir = Path(tmp_dir, "scratch")
        scratch_dir.mkdir()
        
        with input_dir.joinpath("input.fasta").open("wt") as fout:
            for tup in input_sequences.itertuples():
                fout.write(f">{tup.unique_id}\n{tup.protein_sequence}\n")
        
        system_command = f"mmseqs easy-cluster --min-seq-id {min_seq_id} '{input_dir}/input.fasta' '{output_dir}/result' '{scratch_dir}'"
        print(system_command)
        
        proc = subprocess.run(shlex.split(system_command), capture_output=True, check=True)
        
        cluster_df = pd.read_csv(output_dir.joinpath("result_cluster.tsv"), sep="\t", names=["cluster_id", "unique_id"])
        assert len(cluster_df) == len(cluster_df["unique_id"].unique())

    return cluster_df

In [ ]:
input_sequences = sequence_df.merge(final_df[["unique_id"]].drop_duplicates())

len(input_sequences)

In [ ]:
cluster_df = obtain_clusters(input_sequences)

In [ ]:
cluster_df.head()

In [ ]:
if "cluster_id" in final_df:
    del final_df["cluster_id"]

final_df = final_df.merge(cluster_df, on="unique_id", how="outer", validate="m:1")
assert final_df["cluster_id"].notnull().all()

### Load data

In [ ]:
from sklearn.model_selection import GroupKFold, PredefinedSplit
import lightgbm as lgb

In [ ]:
columns_to_drop = [
    "unique_id", "dataset", "name", "mutation", "effect", "effect_type", "foldx_score", "provean_score", "elaspic_score", "cluster_id",
    "test_fold",
]

columns_to_drop += [
    "rosetta_opt_apart_dg_change",
    "rosetta_apart_dg_change",
    "rosetta_complex_dg_change",
    "rosetta_opt_bind_dg_change",
    "rosetta_bind_dg_change",
]

tup = next(final_df.itertuples())
columns_to_drop += [
    field for field in tup._fields if isinstance(getattr(tup, field), (list, tuple, np.ndarray))
]
columns_to_drop

In [ ]:
def get_group(df):
    vc = df["unique_id"].value_counts()
    groups = np.array([vc[uid] for uid in df["unique_id"].unique()])
    return groups

In [ ]:
test_df.head()

In [ ]:
key_columns = [
    "unique_id", "dataset", "name", "mutation", "effect_type", "effect", 
]

eval_columns = [
    "ddg_pred",
    "provean_score",
    "foldx_score",
    "elaspic_score",
    "rosetta_opt_apart_dg_change",
    "rosetta_apart_dg_change",
    "rosetta_complex_dg_change",
    "rosetta_opt_bind_dg_change",
    "rosetta_bind_dg_change",
]

In [ ]:
import heapq
from dataclasses import dataclass, field
from typing import Any

def map_to_test_fold(df):

    @dataclass(order=True)
    class PrioritizedItem:
        priority: int
        idx: int=field(compare=False)
        data: Any=field(compare=False)

    ddg_df = df[df["effect_type"] == "ΔΔG"]
    other_df = df[df["effect_type"] != "ΔΔG"]
    assert len(ddg_df) + len(other_df) == len(df)
    
    ddg_pq = [PrioritizedItem(0, i, []) for i in range(10)]
    for cluster_id, gp in ddg_df.groupby("cluster_id"):
        item = heapq.heappop(ddg_pq)
        item.priority += len(gp)
        item.data.append(cluster_id)
        heapq.heappush(ddg_pq, item)
    
    mapping = {}
    for item in ddg_pq:
        for cluster_id in item.data:
            mapping[cluster_id] = item.idx
    
    del ddg_pq
    
    other_pq = [PrioritizedItem(0, i, []) for i in range(10)]
    for cluster_id, gp in other_df.groupby("cluster_id"):
        if cluster_id in mapping:
            item_idx = mapping[cluster_id]
            item = next(item for item in other_pq if item.idx == item_idx)
            item.priority += len(gp)
            item.data.append(cluster_id)
            heapq.heapify(other_pq)
        else:
            item = heapq.heappop(other_pq)
            item.priority += len(gp)
            item.data.append(cluster_id)
            heapq.heappush(other_pq, item)

    for item in other_pq:
        for cluster_id in item.data:
            if cluster_id in mapping:
                assert mapping[cluster_id] == item.idx
            else:
                mapping[cluster_id] = item.idx
                
    return mapping

In [ ]:
cluster_id_to_test_fold_mapping = map_to_test_fold(final_df)
final_df["test_fold"] = final_df["cluster_id"].map(cluster_id_to_test_fold_mapping)
assert final_df["test_fold"].notnull().all()
assert len(final_df["test_fold"].unique()) == 10

In [ ]:
def get_label(df):
    effect = df["effect"].values
    effect[df["effect_type"] != "ΔΔG"] *= 3
    effect = np.clip(effect, -5, 5) * 1000 + 5000
    return effect

In [ ]:
param = {
    #
    "objective": "lambdarank",
#     "objective": "rank_xendcg",
    "metric": "ndcg",
    "eval_at": 1_000_000,
    "label_gain": [(np.log2(i + 1) + 1) for i in range(0, 10_001)],
    #
    "max_bin": 255,
#     "num_trees": 100,  # aka num_boost_round
    "learning_rate": 0.1,
}

result_dfs = []

ps = PredefinedSplit(final_df["test_fold"])
for train, test in ps.split():
    train_df = final_df.iloc[train].copy()
    test_df = final_df.iloc[test].copy()
    assert not set(train_df["cluster_id"]) & set(test_df["cluster_id"])

    train_ds = lgb.Dataset(
        train_df.drop(columns_to_drop, axis=1),
        label=get_label(train_df),
        group=get_group(train_df),
    )

    valid_ds = lgb.Dataset(
        test_df.drop(columns_to_drop, axis=1),
        label=get_label(test_df),
        group=get_group(test_df),
        reference=train_ds,
    )
    
    bst = lgb.train(param, train_ds, num_boost_round=100, valid_sets=[valid_ds], verbose_eval=10000)
    
    test_df["ddg_pred"] = bst.predict(test_df.drop(columns_to_drop, axis=1), num_iteration=bst.best_iteration)
    result_dfs.append(test_df[key_columns + eval_columns])

In [ ]:
bst.best_score

In [ ]:
result_df = pd.concat(result_dfs, ignore_index=True)
result_df["provean_score"] = -result_df["provean_score"]

In [ ]:
result_df.head()

In [ ]:
def print_spearman_stats(df, feature_columns, target_column):
    df = df.dropna(subset=feature_columns + [target_column])
    for column in feature_columns:
        corr = stats.spearmanr(df[column], df[target_column])
        print(f"{column:30s} {corr[0]:+.4} {corr[1]:.4}")

In [ ]:
print_spearman_stats(result_df, eval_columns, "effect")

In [ ]:
print_spearman_stats(result_df[result_df["effect_type"] == "ΔΔG"], eval_columns, "effect")

In [ ]:
def compute_per_sequence_stats(df, feature_columns, target_column, min_gp_size=6):
    df = df.dropna(subset=feature_columns + [target_column])
    results = {c: [] for c in feature_columns}
    for _, gp in df.groupby("unique_id"):
        if len(gp) < min_gp_size:
            continue
        for column in feature_columns:
            corr = stats.spearmanr(gp[column], gp[target_column])
            results[column].append(corr[0])
    return results

In [ ]:
per_sequence_stats = compute_per_sequence_stats(result_df, eval_columns, "effect")

In [ ]:
fg, ax = plt.subplots()

out = ax.boxplot(per_sequence_stats.values())
_ = ax.set_xticklabels(per_sequence_stats.keys(), rotation="vertical")
# ax.set_ylim(-1, 1)
# fg.tight_layout()

In [ ]:
per_sequence_stats_ddg = compute_per_sequence_stats(
    result_df[result_df["effect_type"] == "ΔΔG"],
    eval_columns,
    "effect",
    18
)

fg, ax = plt.subplots()

out = ax.boxplot(per_sequence_stats_ddg.values())
_ = ax.set_xticklabels(per_sequence_stats_ddg.keys(), rotation="vertical")
# ax.set_ylim(-1, 1)
# fg.tight_layout()

In [ ]:
per_sequence_stats_ddg = compute_per_sequence_stats(
    result_df[result_df["effect_type"] == "ΔΔG"],
    eval_columns,
    "effect",
    2
)

fg, ax = plt.subplots()

out = ax.boxplot(per_sequence_stats_ddg.values())
_ = ax.set_xticklabels(per_sequence_stats_ddg.keys(), rotation="vertical")
# ax.set_ylim(-1, 1)
# fg.tight_layout()

In [ ]:
out.keys()

In [ ]:
palette = ['r', 'g', 'b', 'y']
for x, val, c in zip(xs, vals, palette):
    plt.scatter(x, val, alpha=0.4, color=c)
plt.show()

In [ ]:
train_df[(train_df["effect"] * 1_000).astype(np.int) > 300_000]

In [ ]:
import matplotlib.pyplot as plt

_ = plt.hist(final_df["effect"], bins=100, range=(-5, 5))

In [ ]:
param = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "ndcg_eval_at": 1000000000000,
    "max_bin": 255,
}


bst = lgb.train(param, train_ds, num_boost_round=100, valid_sets=[valid_ds])

In [ ]:
ypred = bst.predict(test_df.drop(columns_to_drop, axis=1), num_iteration=bst.best_iteration)

In [ ]:
ypred = bst.predict(test_df.drop(columns_to_drop, axis=1), num_iteration=bst.best_iteration)
test_df = test_df.copy()
test_df["ddg_pred"] = ypred

In [ ]:
stats.spearmanr(test_df["effect"], test_df["ddg_pred"])

In [ ]:
stats.spearmanr(test_df["effect"], test_df["foldx_score"])

In [ ]:
stats.spearmanr(test_df["effect"], test_df["provean_score"])